# **630710729 นางสาวภารัณ พระสียา**

# **เตรียมข้อมูลหลัก (Master Dataset)**

## **1.1 นำเข้าไลบรารี pandas และอ่านข้อมูลหลักของไฟล์ 'student.csv'**

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('student.csv')

In [ ]:
df.shape

In [ ]:
df.head(20)

## **1.2 แสดงผลข้อมูล&ปัญหาที่พบในข้อมูล**

In [ ]:
print("="*50)
print("🎯 ข้อมูลหลัก (Master Dataset) ")
print("="*50)
print(df)
print("\n📊 สรุปปัญหาที่พบในข้อมูล:")
print(f"- ค่าหายทั้งหมด: {df.isnull().sum().sum()} ค่า")
print(f"- อายุผิดปกติ: {df[df['อายุ'] > 100]['อายุ'].values if any(df['อายุ'] > 100) else 'ไม่มี'}")
print(f"- GPA ผิดปกติ: {df[(df['GPA'] < 0) | (df['GPA'] > 4)]['GPA'].values if any((df['GPA'] < 0) | (df['GPA'] > 4)) else 'ไม่มี'}")
print(f"- รูปแบบการเขียนคณะ: {df['คณะ'].nunique()} แบบ")
print(f"- รูปแบบการเขียนเพศ: {df['เพศ'].nunique()} แบบ")

In [ ]:
#เก็บสำเนาข้อมูลต้นฉบับเพื่อไม่ให้กระทบข้อมูลต้นฉบับ
df_original =df.copy()

## **1.3 วิเคราะห์ข้อมูลเบื้องต้น (EDA)**

- วิเคราะห์ข้อมูล df ที่อ่านเข้ามา

In [ ]:
print("="*50)
print("📋 ข้อมูลทั่วไปของ Dataset")
print("="*50)
df.info()

print("\n" + "="*50)
print("📊 สถิติเชิงพรรณนา")
print("="*50)
print(df.describe())

- ตรวจสอบค่าหายในแต่ละคอลัมน์

In [ ]:
missing_counts = df.isnull().sum()
print("\n❌ ค่าหายในแต่ละคอลัมน์:")
for col, count in missing_counts[missing_counts > 0].items():
    percentage = (count / len(df)) * 100
    print(f"   - {col}: {count} ค่า ({percentage:.1f}%)")

# **เทคนิคจัดการข้อมูล**

## **2.1 จัดการข้อมูลหาย**

-  วิธีที่ 1 ลบแถวที่มีค่าหาย

In [ ]:
# แสดงข้อมูลก่อนลบ
print("ก่อนลบ")
print(df)

# เหมาะเมื่อมีข้อมูลหายน้อย - ลบแถวที่มีค่าหาย
df_clean = df.dropna()

print(f"\nผลการลบ - ก่อนลบ {len(df)} แถว, หลังลบ {len(df_clean)} แถว")
print("หลังลบ (เหลือ 9 คน):")
print(df_clean)

- วิธีที่ 2 เติมค่าด้วยค่าเฉลี่ย

In [ ]:
mean_age = df['อายุ'].mean()
df['อายุ'].fillna(mean_age, inplace=True)

In [ ]:
df

decription: ใช้ค่าเฉลี่ยมาเติมลงในอายุที่หายไป(NaN)

- วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด

In [ ]:
df['คณะ'].fillna(df['คณะ'].mode()[0], inplace=True)

In [ ]:
print("\nหลังเติมค่าใหม่")
print(df)

## **2.2 ทำให้ข้อมูลเป็นมาตรฐาน**

In [ ]:
def standardize_faculty(name):
    if name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์']:
        return 'วิทยาศาสตร์'
    elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts']:
        return 'ศิลปศาสตร์'
    elif name in ['วิศวะ']:
        return 'วิศวกรรมศาสตร์',
    else:
        return name

# ใช้งาน
df['คณะ_มาตรฐาน'] = df['คณะ'].apply(standardize_faculty)

In [ ]:
df

## **2.3 การจัดการค่าผิดปกติ**

In [ ]:
Q1 = df['อายุ'].quantile(0.25)
Q3 = df['อายุ'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# กำหนดขอบเขต
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
# กรองค่าผิดปกติ
df_no_outliers = df[(df['อายุ'] >= lower_bound) & (df['อายุ'] <= upper_bound)]

In [ ]:
print(f"\nขอบเขตล่าง - {lower_bound}")
print(f"ขอบเขตบน - {upper_bound}")
print(f"\nข้อมูลหลังกรองค่าผิดปกติ")
print(df_no_outliers)
print(f"\nค่าเฉลี่ยอายุ (หลังกรองค่าผิดปกติ) - {df_no_outliers['อายุ'].mean():.2f}")

decription: อายุนักศึกษาที่มีค่าผิดปกติ = ใช้ IQR (Interquartile Range)

- แก้ไข GPA ผิดปกติ

In [ ]:
df_no_outliers.loc[df_no_outliers['GPA'] < 0, 'GPA'] = np.nan
df_no_outliers.loc[df_no_outliers['GPA'] > 4, 'GPA'] = 4.0

mean_gpa = round(df_no_outliers['GPA'].mean(), 2)  # ปัดเป็น 2 ทศนิยม
df_no_outliers['GPA'].fillna(mean_gpa, inplace=True)

# ปัดทั้งคอลัมน์เป็น 2 ทศนิยม
df_no_outliers['GPA'] = df_no_outliers['GPA'].round(2)

In [ ]:
df_no_outliers

# **Feature Engineering - การสร้างคุณลักษณะใหม่**

## **One-Hot Encoding แปลงข้อมูลประเภทเป็นตัวเลข**

In [ ]:
data_encoded = pd.get_dummies(df, columns=['คณะ'])
print(data_encoded)

## **สร้าง Feature จากวันเวลา = สร้างสถานะชั่วโมงเรียน**

In [ ]:
bins = [0, 15, 25, 150] # 150 คือค่าสูงสุดที่สมมติขึ้น
labels = ['น้อย', 'ปานกลาง', 'มาก']
df['สถานะชั่วโมงเรียน'] = pd.cut(df['ชั่วโมงเรียน_ต่อสัปดาห์'], bins=bins, labels=labels, right=False)
df


decription: วิเคราะห์จำนวนชั่วโมงเรียนต่อสัปดาห์ โดยแบ่งออกเป็น 3 ช่วง:
1. น้อย ถ้าชั่วโมงเรียนน้อยกว่า 15 ชั่วโมงต่อสัปดาห์
2.   ปานกลาง ถ้าชั่วโมงเรียนอยู่ระหว่าง 15-25 ชั่วโมงต่อสัปดาห์
3.   มาก ถ้าชั่วโมงเรียนมากกว่า 25 ชั่วโมงต่อสัปดาห์

## **การรวม Features = สร้าง BMI จากส่วนสูงและน้ำหนัก**

In [ ]:
df['BMI'] = df['น้ำหนัก_kg'] / (df['ส่วนสูง_cm']/100)**2
df['สุขภาพ'] = pd.cut(df['BMI'],
                                   bins=[0, 18.5, 25, 30, 100],
                                   labels=['ผอม', 'ปกติ', 'อ้วน', 'อ้วนมาก'])

print(df[['ชื่อ', 'BMI', 'สุขภาพ']])